## 資料輸入及資料轉換

In [1]:
import pandas as pd
import numpy as np
import math

X = float(input('Strike Price: '))
S = float(input('Stock Price: '))
r = float(input('Continuously Compounded Risk Rate per Period: '))
u = float(input('Up Factor: '))
d = float(input('Down Factor: '))
n = int(input('Periods: '))

R = math.exp(r)
p = (R-d)/(u-d)

Strike Price: 160
Stock Price: 150
Continuously Compounded Risk Rate per Period: 0.18232
Up Factor: 1.5
Down Factor: 0.5
Periods: 3


# Binomial Process for Stock Price
### 利用 NumPy array 儲存stock price資訊

In [2]:
StockPrice = np.zeros((n+1, n))


StockPrice[0][0] = X * u 
StockPrice[1][0] = X * d


for i in range (0,n+1): 
    for j in range (1,n): 
        if i == j+1: 
            StockPrice[i][j] = StockPrice[i-1][j-1] * d 
        else: 
            StockPrice[i][j] = StockPrice[i][j-1] * u 
print("Binomial tree for stock price: \n", StockPrice)

Binomial tree for stock price: 
 [[240. 360. 540.]
 [ 80. 120. 180.]
 [  0.  40.  60.]
 [  0.   0.  20.]]


## Binomial Probability for Stock Price

In [3]:
StockProb = np.zeros((n+1, n))

StockProb[0][0] = round(p, 3)
StockProb[1][0] = round(1-p, 3)

for i in range (0,n+1): 
    for j in range (1,n): 
        if i == 0:
            StockProb[i][j] = round(StockProb[i][j-1] * p, 3)
        else:
            StockProb[i][j] = round(StockProb[i][j-1] * p + StockProb[i-1][j-1] * (1-p), 3)
print("Binomial probability for stock price: \n", StockProb)

Binomial probability for stock price: 
 [[0.7   0.49  0.343]
 [0.3   0.42  0.441]
 [0.    0.09  0.189]
 [0.    0.    0.027]]


# Binomial Process for Call Price
### 一樣使用 NumPy array，利用 backward induction，因此先處理 leaf 的資訊

In [4]:
CallPrice = np.zeros((n+1, n))
for i in range (0,n+1):
    if (StockPrice[i][n-1] - S) > 0:
        CallPrice[i][n-1] = StockPrice[i][n-1] - S
    else:
        CallPrice[i][n-1] = 0

### 接下來，一步一步回推得到 call price

In [5]:
for j in range (0,n-1): 
    for i in range (0,n):
        CallPrice[i][n-j-2] = round((p * CallPrice[i][n-j-1] + (1-p) * CallPrice[i+1][n-j-1]) / R, 3)
print("Binomial tree for call price: \n", CallPrice)
print("\nFinal Call Price: ", (p * CallPrice[0][0] + (1-p) * CallPrice[1][0])/R )        

Binomial tree for call price: 
 [[141.458 235.    390.   ]
 [ 10.208  17.5    30.   ]
 [  0.      0.      0.   ]
 [  0.      0.      0.   ]]

Final Call Price:  85.06909477256853


# Binomial Process for Put Price
### 步驟與 Call Price 相同

In [6]:
PutPrice = np.zeros((n+1, n))
for i in range (0,n+1):
    if (S - StockPrice[i][n-1]) > 0:
        PutPrice[i][n-1] = S - StockPrice[i][n-1]
    else:
        PutPrice[i][n-1] = 0

In [7]:
for j in range (0,n-1): 
    for i in range (0,n):
        PutPrice[i][n-j-2] = round((p * PutPrice[i][n-j-1] + (1-p) * PutPrice[i+1][n-j-1]) / R, 3)
print("Binomial tree for put price: \n", PutPrice)
print("\nFinal Put Price: ", (p * PutPrice[0][0] + (1-p) * PutPrice[1][0])/R )     

Binomial tree for put price: 
 [[  5.625   0.      0.   ]
 [ 34.375  22.5     0.   ]
 [ 49.583  85.     90.   ]
 [  0.      0.    130.   ]]

Final Put Price:  11.87506324480364


# 輸出結果：Call Price 及 Put Price

In [8]:
print("Call Price: ", (p * CallPrice[0][0] + (1-p) * CallPrice[1][0])/R)
print("Put  Price: ", (p * PutPrice[0][0] + (1-p) * PutPrice[1][0])/R)

Call Price:  85.06909477256853
Put  Price:  11.87506324480364
